In [2]:
!pip install imbalanced-learn

  Obtaining dependency information for imbalanced-learn from https://files.pythonhosted.org/packages/a3/9e/fbe60a768502af54563dcb59ca7856f5a8833b3ad5ada658922e1ab09b7f/imbalanced_learn-0.11.0-py3-none-any.whl.metadata
  Using cached imbalanced_learn-0.11.0-py3-none-any.whl.metadata (8.3 kB)
  Obtaining dependency information for scikit-learn>=1.0.2 from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached imbalanced_learn-0.11.0-py3-none-any.whl (235 kB)
Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1


In [4]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import os
import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

In [24]:
df = pd.read_csv('local_file.csv')

In [25]:
df['step'] = df['step'] / 24

In [26]:
max_value = df['step'].max()
min_value = df['step'].min()

print(f'Highest Value: {max_value}')
print(f'Lowest Value: {min_value}')

Highest Value: 30.958333333333332
Lowest Value: 0.041666666666666664


In [27]:
df['step'] = pd.to_numeric(df['step'], errors='coerce')

In [28]:
bin_edges = [0, 12, 24, 36]
bin_labels = ['Normal', 'Whole', 'Stay']
df['step'] = pd.cut(df['step'], bins=bin_edges, labels=bin_labels, right=False)

In [29]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['step'] = label_encoder.fit_transform(df['step'])


In [30]:
df['type'] = df['type'].replace(['TRANSFER', 'DEBIT'], 'OTHER')

In [31]:
frequency_map = df['type'].value_counts(normalize=False).to_dict()
df['type'] = df['type'].map(frequency_map)

In [32]:
columns_to_drop = ['nameOrig', 'nameDest', 'isFlaggedFraud']
df= df.drop(columns=columns_to_drop)

In [33]:
columns_to_drop = ['oldbalanceDest', 'oldbalanceOrg']
df = df.drop(columns=columns_to_drop)

In [34]:
X= df.drop(columns=['isFraud'])
y=df['isFraud']

In [35]:
df.head()

,step,type,amount,newbalanceOrig,newbalanceDest,isFraud
0,0,2151495,9839.64,160296.36,0.0,0
1,0,2151495,1864.28,19384.72,0.0,0
2,0,574341,181.00,0.00,0.0,1
3,0,2237500,181.00,0.00,0.0,1
4,0,2151495,11668.14,29885.86,0.0,0


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state = 42)

In [37]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [38]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)

In [39]:
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [40]:
# Convert the resampled data back to Pandas DataFrames
X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
y_resampled_df = pd.DataFrame(y_resampled, columns=['isFraud'])
y_test_df = pd.DataFrame(y_train, columns=['isFraud'])

# Save to CSV
X_resampled_df.to_csv('X_train_resampled_svm.csv', index=False)
y_resampled_df.to_csv('y_train_resampled_svm.csv', index=False)
X_test.to_csv('X_test_svm.csv', index=False)
y_test.to_csv('y_test_svm.csv', index=False)

In [41]:
# Initialize a boto3 client
s3_client = boto3.client('s3')

bucket = 'udacity-nano-degree-project'
prefix = 'processed_data_svm'
output_prefix = 'svm-output'

# Upload the files
s3_client.upload_file('X_train_resampled_svm.csv', bucket, f'{prefix}/train/X_train_resampled.csv')
s3_client.upload_file('y_train_resampled_svm.csv', bucket, f'{prefix}/train/y_train_resampled.csv')
s3_client.upload_file('X_test_svm.csv', bucket, f'{prefix}/test/X_test_svm.csv')
s3_client.upload_file('y_test_svm.csv', bucket, f'{prefix}/test/y_test_svm.csv')

In [42]:
# Now your data is in S3 and you can set up the s3_input objects
s3_input_train = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/train/', content_type='csv')
s3_input_test = sagemaker.inputs.TrainingInput(s3_data=f's3://{bucket}/{prefix}/test/', content_type='csv')

In [ ]:
from datetime import datetime

# Set your SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()  # Make sure this IAM role has the necessary permissions

# Create a unique name for the estimator
unique_name = f"SMOTE-SVM-Estimator-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"

# Create SKLearn estimator
sklearn_estimator = SKLearn(
    entry_point='svm.py', # Path to your training script
    role=role,
    instance_type='ml.m5.xlarge',  # Example instance type, choose one based on your needs
    framework_version='0.23-1',  # Match this to your scikit-learn version
    py_version='py3',  # Choose Python version
    output_path=f's3://{bucket}/{output_prefix}',
    sagemaker_session=sagemaker_session,
    base_job_name=unique_name
)

# Fit estimator
sklearn_estimator.fit({'train': s3_input_train, 'test': s3_input_test})

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: SMOTE-SVM-Estimator-2023-11-10-02-10-36-2023-11-10-02-10-36-751


2023-11-10 02:10:37 Starting - Starting the training job...
2023-11-10 02:10:52 Starting - Preparing the instances for training......
2023-11-10 02:12:07 Downloading - Downloading input data......
2023-11-10 02:13:03 Training - Training image download completed. Training in progress..2023-11-10 02:13:04,581 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-11-10 02:13:04,584 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-10 02:13:04,628 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-11-10 02:13:04,801 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-10 02:13:04,813 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-10 02:13:04,825 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-11-10 02:13:04,834 sagemaker-training-toolkit INFO     Invoking user sc